<a href="https://colab.research.google.com/github/Renato89/rain-in-australia/blob/main/RainInAustralia%5Bcon_ETL_e_SARIMA%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Piogge in Australia


## Preparazione dell'ambiente di sviluppo

In [ ]:
%%capture
!pip install geopandas

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import math
import missingno
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import re
import geopandas
import folium
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from numpy import log

from google.colab import drive

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Esplorazione dei dati

In [ ]:
df = pd.read_csv('data/weatherAUS.csv')
df.info()

In [ ]:
df

Un' oservazione al giorno per un periodo di quasi dieci anni, raggruppati per località. 

Il seguente listato recupera le informazioni geografiche per ognuna delle località e le visualizza su una mappa.
La libreria usata si chiama geopandas e si basa sui DataFrame di pandas ma con l'aggiunta di un campo che contiene informazioni geometriche del record.
Inoltre, si sfrutta la libreria folium per visualizzare le località su una mappa interattiva.

In [ ]:
app = Nominatim(user_agent="renato")

lat = []
lon = []
loc = []
not_found = []
for loc_name in df['Location'].unique():
    formatted = re.sub("([a-z])([A-Z])","\g<1> \g<2>", loc_name)
    display_name = formatted + ', Australia'
    location = app.geocode(display_name)
    if location != None:
        lat.append(location.raw['lat'])
        lon.append(location.raw['lon'])
        loc.append(loc_name)
    else:
        not_found.append(loc_name)

d = {'Location':loc, 'Latitude':lat, 'Longitude':lon}
geolocations = pd.DataFrame(data=d)

gdf = geopandas.GeoDataFrame(
    geolocations, 
    geometry=geopandas.points_from_xy(geolocations.Longitude, geolocations.Latitude))

print("Località non trovate: ", len(not_found))

In [ ]:
m = folium.Map(location=[-23, 133], zoom_start=4.4, tiles='CartoDB positron')

for _, r in gdf.iterrows():
    lat = r['geometry'].y
    lon = r['geometry'].x
    folium.Marker(location=[lat, lon],
                  popup='{} <br> lat: {} <br> lon: {}'.format(r['Location'], lat, lon)).add_to(m)

m

#Pulizia dei dati

Il primo problema da affrontare è conoscere la quantità dei dati mancanti. Per avere un'idea chiara si possono sfruttare strumenti di visualizzazione come ciò che mette a disposizione la libreria missingno. Questa mostra in una matrice grafica la distribuzione dei valori nulli di tutte le colonne.

In [ ]:
if df.isnull().any(axis=None):
    missingno.matrix(df)

Per avere un'idea di massima si può visualizzare l'istogramma che mostra il conteggio delle osservazioni per ogni giorno.

In [ ]:
fig = sns.catplot(x='Date', kind='count', data=df, height=6, aspect=2)
fig.set(xticklabels=[], title='Conteggio delle osservazioni rispetto al campo data')

Il grafico mostra il conteggio delle osservazioni per ogni campo data della tabella. Infatti, per lo stesso giorno, abbiamo l'osservazioni di diverse stazioni meteorologiche dislocate in 49 città.
Si nota dal grafico come un porzione iniziale ed una buona fetta nella parte finale del periodo di osservazione, soffrano di mancanza dati. Inoltre, si può vedere che circa la metà della serie abbia una porzione piccola di località che possiede poche osservazioni. Si intervine su questi due aspetti per "ritagliarsi" un porzione contiugua della serie che possieda una bassa percentuale di valori nulli.



## Rimozione di record non validi
Da queste informazioni, possiamo scegliere di individuare ed escludere quelle località e quei periodi dal dataset in modo da avere un notevole miglioramento nella qualità dei dati.

Iniziamo col tagliare via le date all'inizio ed alla fine del periodo di osservazione che possiedono pochissime osservazioni. Si può vedere che queste hanno un conteggio inferiore a 30. Queste, trovandosi all'estremità del periodo, non inficierà sul modello della serie.

In [ ]:
valid_records = df['Date'].value_counts() > 30
filter = [ valid_records[x]  for x in df['Date']]

truncated_df = df[filter]

print("Numero di osservazioni rimosse: ", len(df) - len(truncated_df))

A questo punto cerchiamo le località che hanno mancanza di dati.

In [ ]:
fig = sns.catplot(x='Location', kind='count', data=truncated_df, height=6, aspect=2, order=df['Location'].value_counts().index)
fig.set_xticklabels(rotation=90)
fig.set(title='Conteggio osservazioni per località')

Tre località possiedo metà dei dati rispetto alle altre 46. Queste sono: Katherine, Nhil e Uluru. Decidiamo di eliminarle. Eliminandole, le escludiamo dal modello predittivo, possiamo farlo perchè abbiamo a disposizione una serie temporale per ogni località, quindi, ne abbiamo a disposizione ancora 46.


In [ ]:
filtered_df = truncated_df[(truncated_df['Location'] != 'Katherine' ) & 
                           (truncated_df['Location'] != 'Nhil' ) & 
                           (truncated_df['Location'] != 'Uluru' )]
print("Numero di osservazioni rimosse: ", len(truncated_df) - len(filtered_df))

Verifichiamo se il "ritaglio" è andato a buon fine.

In [ ]:
fig = sns.catplot(x='Date', kind='count', data=filtered_df, height=6, aspect=2)
fig.set(xticklabels=[], title='Conteggio delle osservazioni rispetto al campo data')

# Predizione Univariata 
Si vuole prevedere l'arrivo della pioggia. Estraiamo la colonna in esame, 'RainFall' e eseguiamo la procedura per ottenere un modello SARIMA dalla serie. 


In [ ]:
rainfall_df = filtered_df[['Date','Location','Rainfall']]
print(rainfall_df)

## Imputazione
Avendo a disposizione molte località da cui sono state prese le misure, se una località possiede un valore nullo, con buona probabilità il valore catturato da una stazione vicina non è nullo e vicino a quello che sarebbe stato il valore reale.
Quindi, si riordina la serie per data, in modo tale da avere vicini le osservazioni dello stesso giorno, e poi si esegue un'imputazione semplice, assegnando il valore più vicino valido quando si incontra un dato mancante.

In [ ]:
rainfall_sorted = rainfall_df.sort_values(by='Date')
print(rainfall_sorted)

In [ ]:
last = 0.0
imputed_vals = []
for _, r in rainfall_sorted.iterrows():
    current = r['Rainfall']

    if pd.isnull(current):
        imputed_vals.append(last)
    else:
        imputed_vals.append(current)
        last = current

d = {'Date': rainfall_sorted['Date'],
     'Location': rainfall_sorted['Location'], 
     'Rainfall': imputed_vals}
imputed_rainfall = pd.DataFrame(data=d).sort_values(by='Location')

Si ricontrolla sei i valori nulli sono stati rimossi.

In [ ]:
before = (rainfall_df.isnull().mean()*100).rename('Prima')
after = (imputed_rainfall.isnull().mean()*100).rename('Dopo')

null_count = before.to_frame().merge(after.rename('Dopo'), left_index=True, right_index=True)
print(null_count)

## Date mancanti
Nonostante i valori nulli non siano più presenti può sempre essere presente il problema di date mancanti. Per effettuare le predizioni è necessario che ogni riga rappresenti lo stesso intervallo di tempo, nel nostro caso un giorno. Quindi, si esegue la seguente procedura:


1.   si individua l'intervallo temporale della serie
2.   si genera una lista di indici temporali con frequenza giornaliera
3.   la si confronta con quella del dataset e, se non corrisponde,
4.   si espande la serie temporale con le dati mancanti aggiungendo dei valori 
nulli
5.   si estraggono gli stessi periodi mancanti ma dell'anno precedente e,
6.   si sostituiscono con essi.


In [ ]:
location = 'Melbourne'
filtered = imputed_rainfall.query('Location == @location')
rainfall_series = filtered['Rainfall']
rainfall_series.index = pd.to_datetime(filtered['Date'])
rainfall_series.sort_index()

In [ ]:
first_day = rainfall_series.index.min()
last_day = rainfall_series.index.max()
print("Finestra temporale da", first_day, " a ", last_day) 

In [ ]:
correct_index = pd.period_range(first_day, last_day, freq = 'D')
actual_index = rainfall_series.index.to_period(freq = 'D')
print("Numero di giorni del periodo: ", len(correct_index))
print("Numero di record nei dati: ", len(actual_index))

In [ ]:
rainfall_series.index = actual_index
expanded_df = rainfall_series.reindex(correct_index).to_frame()

In [ ]:
missingno.matrix(expanded_df, figsize=(5,10))

Come si può vedere, i vuoti nelle date si concentrano i tre punti. In realtà, se si va nel dettaglio, si troverà che tale date corrispondono a precisamente tre mesi.


In [ ]:
# Estrai gli intervalli di tempo mancanti 

null_intervals = []
counting = False
series = expanded_df['Rainfall']
first = series.index[0]
last = first
for idx, x in series.iteritems():

    if math.isnan(x) and not counting:
        first = idx
        counting = True
    
    if not math.isnan(x) and counting:
        last = idx - 1
        counting = False
        null_intervals.append(pd.period_range(first,last))

print("Intervalli temporali: \n", null_intervals)

In [ ]:
# Sostituisci il periodo mancante con lo stesso periodo dell'anno precedente

for p in null_intervals:
    start_day = p[0].day
    start_month = p[0].month
    start_year = p[0].year - 1 # anno precedente

    end_day = p[-1].day
    end_month = p[-1].month
    end_year = p[-1].year - 1 # anno precedente

    prev_year_start = pd.Period(year=start_year, month=start_month, day=start_day, freq='D')
    prev_year_end = pd.Period(year=end_year, month=end_month, day=end_day, freq='D')

    prev_year = pd.period_range(prev_year_start, prev_year_end)

    # sostituisci
    series.loc[p] = series.loc[prev_year].array 

In [ ]:
print("Valori null dopo la modifica: ", series.isnull().mean()*100)

# Il modello
Si sceglie il modello SARIMA. Prendiamo una località, per esempio Sydney e verifichiamo la stazionarietà della serie.

## Scelta del parametro di differenziazione


In [ ]:
data = series.to_frame().sort_index()
data['D.Rainfall'] = data['Rainfall'].diff()

In [ ]:
data.plot(kind='line',subplots=True, figsize=(15,10))

In [ ]:
pv = adfuller(data['Rainfall'])[1]
print('p-value: %f' % pv)

Visto che il test di Fuller risulta con un p-value zero, significa che la serie è già stazionaria e non necessita di differenziazione.


## Scelta del parametro di media mobile (MA)


In [ ]:
sm.graphics.tsa.plot_acf(data['Rainfall'].values.squeeze(), lags=40)
plt.show()

Il numero dei primi punti che si trovano al di fuori dell'area di confidenza, indica il numero di termini necessari per rimuovere l'autocorrelazione dalla serie stazionaria.

In questo caso scegliamo un valore di autoregressione (MA) di 1.

## Scelta del parametro di autoregressione (AR)
Si ispeziona i termini precedenti senza considerari quelli nel mezzo.

In [ ]:
sm.graphics.tsa.plot_pacf(data['Rainfall'].values.squeeze(), lags=40, method="ywm")
plt.show()

Si sceglie p=1.

Scomporre il dataset.

In [ ]:
# TODO cpntinuare da qui, il df pronto si trova in data

In [ ]:
fig = plt.figure(figsize=(20,8))
model = ARIMA(data['Rainfall'], order=(1,0,1)) 
ax = plt.gca()
results = model.fit() 
plt.plot(data['Rainfall'])
plt.plot(results.fittedvalues, color='red')
ax.legend(['Rainfa', 'Forecast'])

print(results.summary())

In [ ]:
endog = data.loc[:, "Rainfall"]
nobs = endog.shape[0]

In [ ]:
endog

2009-01-01    0.0
2009-01-02    0.2
2009-01-03    0.6
2009-01-04    0.0
2009-01-05    0.0
             ... 
2017-06-21    0.6
2017-06-22    0.2
2017-06-23    0.0
2017-06-24    1.4
2017-06-25    0.0
Freq: D, Name: Rainfall, Length: 3098, dtype: float64

In [ ]:
filtered.index = pd.period_range('2009-01-01', '2017-06-25', freq = 'D')

Modello su un sotto insieme dei dati

In [ ]:
mod = sm.tsa.statespace.SARIMAX(endog.loc[:'2016'], trend='c' , order=(1,0,1))
fit_res = mod.fit(disp=False, maxiter=250)
print(fit_res.summary())

Modello su tutto il dataset

In [ ]:
mod = sm.tsa.statespace.SARIMAX(endog, trend='c', order=(3,0,2))
res = mod.filter(fit_res.params)

## Predizioni out-of-sample


In [ ]:
print(res.forecast())

In [ ]:
fcast_res2 = res.get_forecast(steps=2)
# Note: since we did not specify the alpha parameter, the
# confidence level is at the default, 95%
print(fcast_res2.summary_frame())

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

# Plot the data (here we are subsetting it to get a better look at the forecasts)
endog.loc['2016':].plot(ax=ax)

# Construct the forecasts
fcast = res.get_forecast().summary_frame()
fcast['mean'].plot(ax=ax, style='k--')
ax.fill_between(fcast.index, fcast['mean_ci_lower'], fcast['mean_ci_upper'], color='k', alpha=0.1);